# *PyThermoML* - Import Template
This template explains how *PyThermoML* can be used, to write data into ThermoML.

**WARNING API is still changig. Only use under reservation.**


Before usage make sure that you've installed json and lxml e. g. via pip.

In [1]:
from core import PureOrMixtureData, Measurement, DataReport, Compound, DataPoint

## DataReport
First create a dataReport instance, in which you can declare the title, the DOI and the authors of the referred paper.


In [3]:
dataReport = DataReport("Title of referred paper", "DOI of referred paper", "author 1", "author 2")

## Compounds
After that you can declare the compounds which you have used in your chemical system.

In [4]:
comp1 = Compound("id1", "inhi1", "inchikey1", "smiles1", "water")
comp2 = Compound("id2", "inchi2", "inchikey2", "smiles2", "ethanol")

The attributes of Compounds are an internal ID, the inchi code, inchi key, smiles code and a common Name. In the next step, add the compounds to dataReport. With the ID you can reference to the compound later.

In [5]:
comp1_ID = dataReport.addCompound(comp1)
comp2_ID = dataReport.addCompound(comp2)

## Pure or mixture data
The main part begins here: Declare in a PureOrMixtureData object which compounds you have used, by calling them by their ID. Give your experiment also an ID and a name.

In [6]:
experiment = PureOrMixtureData("ID", "experiement1", comp1_ID, comp2_ID)

## Declaration of Property and Variable
Declare your examined property (e.g. the viscosity) and variables (e. g. the temperature, mole fraction) of the experiment/simulation.

Concerning the property, enter again an ID and indicate whether you've done a simulation or an experiment.
The variable has also an ID. Be aware, that there are compound specific variables/properties (e. g. mole fraction). The function returns an ID to refer to respective property/variable.

In [7]:
from props.transportproperties import Viscosity
from vars.componentcomposition import MoleFraction
from vars.temperature import Temperature


#property definitions
visc = Viscosity(ID='visc1', method="simulation")
# Variable definitions
temp = Temperature(ID='temp1')
frac1 = MoleFraction(ID='moleFrac1', compoundID=comp1_ID)
frac2 = MoleFraction(ID='moleFrac2', compoundID=comp2_ID)

This API provides the following properties:
* volumetric properties:
    * mass density (kg/m3)
* heat capacity properties:
    * molar heat capacity at constant pressure (J/K/mol)
    * molar heat capacity at constant volume (J/K/mol)
* transport properties:
    * viscosity (Pa*s)
    * kinematic viscosity (m2/s)
    * microviscosity (Pa*s)
    * self dffusion coefficient (m2/s)
* bioproperties:
    * peakTemperature (K)
* other:
    * surface tension (N/m) 
    * speed of sound (m/s)

This API provides the following variables:
* component composition:
    * mole fraction ()
* temperatures:
    * temperature(K)
    * upper temperature(K)
    * lower temperature(K)
* pressure:
    * pressure (kPa)
    

The units are fixed and cannot be changed!
For usage import the respective packages.

Add initialized properties/variables to experiement. The experiment is now ready to get filled with data.


In [8]:
viscID = experiment.addProperty(visc)
tempID = experiment.addVariable(temp)
frac1ID = experiment.addVariable(frac1)
frac2ID = experiment.addVariable(frac2)

## Add data to experiment
Data can be added to experiment by creating a Measurement object. It contains again a name, a values dictionary and a reference to the expereiment. First declare the values dictionary.
Therfore use the ID of the property/variable as key and enter the values.

In [9]:
measurementID = "meas1"

viscDataPoint = DataPoint(
    measurementID=measurementID,
    value=0.001,
    propID=viscID,
    uncertainty=0.1
)

tempDataPoint = DataPoint(
    measurementID=measurementID,
    value=300.0,
    varID=tempID,
    uncertainty=10.0
)

frac1DataPoint = DataPoint(
    measurementID=measurementID,
    value=0.5,
    varID=frac1ID,
    uncertainty=0.01
)

frac2DataPoint = DataPoint(
    measurementID=measurementID,
    value=0.5,
    varID=frac2ID,
    uncertainty=0.02
)

datapoints = [viscDataPoint, tempDataPoint, frac1DataPoint, frac2DataPoint]

In this **example** (T = 300K, mole fraction of water and ethanol = 1/2) the mixture had a viscosity of 0.001 Pa*s.

For each variable and property an uncertainty can be declared. For variables this API stores the **expanded uncertainty**. It is a quantity defining an interval about the result of a measurement that may be expected to encompass a large fraction of the distribution of values that could reasonably be attributed to the measurand. For properties this API stores the **combined standard uncertainty**.

In [10]:
# add Measurement to experiment
experiment.addMeasurement(dataPoints=datapoints)

# add experiment to dataReport
dataReport.addPureOrMixtureData(experiment)
print(dataReport)

{
    "_title": "Title of referred paper",
    "_DOI": "DOI of referred paper",
    "_authors": {
        "_author0": "author 1",
        "_author1": "author 2"
    },
    "compounds": {
        "id1": {
            "_Compound__type": "comp",
            "_ID": "id1",
            "_standardInchi": "inhi1",
            "_standardInchiKey": "inchikey1",
            "_smiles": "smiles1",
            "_commonName": "water"
        },
        "id2": {
            "_Compound__type": "comp",
            "_ID": "id2",
            "_standardInchi": "inchi2",
            "_standardInchiKey": "inchikey2",
            "_smiles": "smiles2",
            "_commonName": "ethanol"
        }
    },
    "pureOrMixtureData": {
        "ID": {
            "_ID": "ID",
            "comps": [
                "experiement1",
                "id1",
                "id2"
            ],
            "_properties": {
                "visc1": {
                    "_propName": "Viscosity",
                    "_ID"

Congratulations your DataReport object has been created sucessfully. Now you can write your DataReport to a ThermoML file by using the writeThermo() function. If you want to modify your ThermoML file you can read it by readThermo()

In [11]:
from tools.readTools import readThermo
from tools.writeTools import writeThermo


writeThermo(dataReport, filename='testThermo')
readThermo(path="testThermo.xml")